In [13]:
import os
import json
from typing import Dict, List, Tuple, Any
from collections import defaultdict
import mido

In [5]:
for (root,dirs,files) in os.walk('./data',topdown=True):
  # print("Directory path: %s"%root)
  # print("Directory Names: %s"%dirs)
  # print("Files Names: %s"%files)
  for file in files:
    print(f'{root}/{file}')

./data/.DS_Store
./data/midiindx.htm
./data/brandenb/brand1.mid
./data/brandenb/brand2.mid
./data/brandenb/brand3.mid
./data/brandenb/brand52.mid
./data/brandenb/brand53.mid
./data/brandenb/brand51.mid
./data/brandenb/brand41.mid
./data/brandenb/brand43.mid
./data/brandenb/brand42.mid
./data/cantatas/jesu2.mid
./data/cantatas/jesu1.mid
./data/sinfon/sinfon14.mid
./data/sinfon/sinfon1.mid
./data/sinfon/sinfon15.mid
./data/sinfon/sinfon3.mid
./data/sinfon/sinfon2.mid
./data/sinfon/sinfon12.mid
./data/sinfon/sinfon6.mid
./data/sinfon/sinfon7.mid
./data/sinfon/sinfon13.mid
./data/sinfon/sinfon11.mid
./data/sinfon/sinfon5.mid
./data/sinfon/sinfon4.mid
./data/sinfon/sinfon10.mid
./data/sinfon/sinfon9.mid
./data/sinfon/sinfon8.mid
./data/suites/airgstr4.mid
./data/organ/passac.mid
./data/organ/prefug8.mid
./data/organ/catechor.mid
./data/organ/catech9.mid
./data/organ/catech8.mid
./data/organ/catech5.mid
./data/organ/trio3b.mid
./data/organ/trio3c.mid
./data/organ/catech4.mid
./data/organ/cat

In [9]:
sample_file = './data/brandenb/brand1.mid'

In [10]:
def parse_midi_to_dict(path: str) -> Dict[str, Any]:
    """
    Parse a .mid file and return a dictionary with:
      - 'ticks_per_beat' (int)
      - 'num_tracks' (int)
      - 'tempos' (List[Tuple[int, int]])            # (abs_tick, microseconds_per_beat)
      - 'time_signatures' (List[Tuple[int, int,int,int]])   # (abs_tick, numerator, denominator, clocks_per_click)
      - 'key_signatures' (List[Tuple[int, str]])    # (abs_tick, key_string)
      - 'tracks' -> Dict[str, List[Tuple[int, int, int]]]   # 'track_0': [(pitch, start_tick, duration_tick), ...], ...
    All start times and durations are in ticks (MIDI pulses), not seconds.
    """
    mid = mido.MidiFile(path)
    ticks_per_beat = mid.ticks_per_beat

    result: Dict[str, Any] = {
        "ticks_per_beat": ticks_per_beat,
        "num_tracks": len(mid.tracks),
        "tempos": [],           # (tick, us_per_beat)
        "time_signatures": [],  # (tick, num, den, clocks_per_click)
        "key_signatures": [],   # (tick, key)
        "tracks": {}
    }

    # For collecting global meta events, we’ll traverse all tracks, keeping per-track absolute tick
    # then merge into the result lists.
    global_tempos: List[Tuple[int, int]] = []
    global_timesigs: List[Tuple[int, int, int, int]] = []
    global_keysigs: List[Tuple[int, str]] = []

    # Per-track: accumulate absolute time and notes
    for t_idx, track in enumerate(mid.tracks):
        abs_tick = 0
        # For overlapping same-pitch notes, keep a stack per pitch
        open_notes: Dict[int, List[int]] = defaultdict(list)  # pitch -> [start_tick, ...]
        finished_notes: List[Tuple[int, int, int]] = []       # (pitch, start, duration)

        for msg in track:
            abs_tick += msg.time  # mido gives delta time in ticks by default
            if msg.is_meta:
                if msg.type == "set_tempo":
                    global_tempos.append((abs_tick, msg.tempo))
                elif msg.type == "time_signature":
                    global_timesigs.append((abs_tick, msg.numerator, msg.denominator, msg.clocks_per_click))
                elif msg.type == "key_signature":
                    global_keysigs.append((abs_tick, msg.key))
                # ignore other meta for this summary
                continue

            # Note events
            if msg.type == "note_on" and msg.velocity > 0:
                open_notes[msg.note].append(abs_tick)
            elif (msg.type == "note_off") or (msg.type == "note_on" and msg.velocity == 0):
                # Close the most recent unmatched note_on for this pitch, if any
                if open_notes[msg.note]:
                    start_tick = open_notes[msg.note].pop()
                    duration = abs_tick - start_tick
                    if duration >= 0:
                        finished_notes.append((msg.note, start_tick, duration))
                # else: unmatched note_off; ignore

        # Optionally drop any still-open notes (unmatched note_on). Could infer duration to EOT, but safer to skip.

        # Sort by start tick to ensure sequence order
        finished_notes.sort(key=lambda x: x[1])

        result["tracks"][f"track_{t_idx}"] = finished_notes

    # Sort and assign global meta lists
    global_tempos.sort(key=lambda x: x[0])
    global_timesigs.sort(key=lambda x: x[0])
    global_keysigs.sort(key=lambda x: x[0])

    result["tempos"] = global_tempos
    result["time_signatures"] = global_timesigs
    result["key_signatures"] = global_keysigs

    return result


In [14]:
data = parse_midi_to_dict(sample_file)
# data["tracks"]["track_0"] -> [(60, 0, 480), (64, 0, 480), ...]
# data["ticks_per_beat"]    -> e.g., 480
# data["tempos"]            -> [(0, 500000), (1920, 600000), ...]
print(json.dumps(data, indent=4))


{
    "ticks_per_beat": 120,
    "num_tracks": 12,
    "tempos": [
        [
            0,
            750000
        ],
        [
            40080,
            1500000
        ],
        [
            40320,
            1500000
        ],
        [
            54360,
            600000
        ],
        [
            83520,
            666666
        ],
        [
            84240,
            600000
        ],
        [
            97560,
            631578
        ],
        [
            97920,
            666666
        ],
        [
            98280,
            705882
        ],
        [
            98640,
            1500000
        ],
        [
            99000,
            444444
        ],
        [
            116640,
            600000
        ],
        [
            137160,
            444444
        ],
        [
            154980,
            1000000
        ],
        [
            166380,
            3000000
        ],
        [
            166500,
            4